# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.data.datapath import DataPath
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azureml.core import Workspace, Dataset
from azureml.train.automl.utilities import get_primary_metrics
from azureml.widgets import RunDetails
from azureml.core.run import Run
from azureml.train.automl.run import AutoMLRun

from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice

from azureml.core.authentication import InteractiveLoginAuthentication
import requests

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

KeyboardInterrupt: 

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-231015
aml-quickstarts-231015
westus2
2c48c51c-bd47-40d4-abbe-fb8eabd19c8c


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "Project-3-Compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 0, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# choose a name for experiment
experiment_name = 'Project_3_Experiment'
experiment=Experiment(ws, experiment_name)

### Upload Data

Must Upload the data manually before running this part

In [8]:
dataset = Dataset.get_by_name(ws, name='behavioral_data')
dataset.to_pandas_dataframe()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,1.0,1.0,1292015,1,29,2015,1200.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0
1,1.0,1.0,1202015,1,20,2015,1100.0,2.015000e+09,2.015000e+09,1.0,...,2.0,2.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN,2.0
2,1.0,1.0,2012015,2,1,2015,1200.0,2.015000e+09,2.015000e+09,1.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN
3,1.0,1.0,1142015,1,14,2015,1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,9.0
4,1.0,1.0,1142015,1,14,2015,1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2.0,11.0,11202015,11,20,2015,1100.0,2.015002e+09,2.015002e+09,1.0,...,2.0,2.0,3.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0
9996,2.0,11.0,11052015,11,5,2015,1100.0,2.015002e+09,2.015002e+09,1.0,...,3.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0
9997,2.0,11.0,11242015,11,24,2015,1100.0,2.015002e+09,2.015002e+09,1.0,...,2.0,2.0,2.0,2.0,3.0,1.0,1.0,NaN,NaN,2.0
9998,2.0,11.0,11142015,11,14,2015,1100.0,2.015002e+09,2.015002e+09,1.0,...,9.0,9.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,9.0


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

Target: GENHLTH - general health

In [9]:
get_primary_metrics('regression')

['r2_score',
 'normalized_root_mean_squared_error',
 'normalized_mean_absolute_error',
 'spearman_correlation']

In [42]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_hours": .5,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'normalized_mean_absolute_error'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "regression",
                             training_data=dataset,
                             label_column_name="GENHLTH",   
                             path = '.',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [43]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Project_3_Experiment,AutoML_bc06420d-c0d5-4168-893b-65dae5508651,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
list_runs = experiment.get_runs()
for run in list_runs:
    print(run.id)

AutoML_bc06420d-c0d5-4168-893b-65dae5508651


In [21]:
remote_run = AutoMLRun(experiment, 'AutoML_bc06420d-c0d5-4168-893b-65dae5508651')

In [22]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
Project_3_Experiment,AutoML_bc06420d-c0d5-4168-893b-65dae5508651,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [23]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [45]:
remote_run.wait_for_completion()

{'runId': 'AutoML_bc06420d-c0d5-4168-893b-65dae5508651',
 'target': 'Project-3-Compute',
 'status': 'Completed',
 'startTimeUtc': '2023-04-13T16:55:21.962125Z',
 'endTimeUtc': '2023-04-13T17:35:59.110471Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_mean_absolute_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'Project-3-Compute',
  'AMLSettingsJsonString': '{"path":null,"name":"Project_3_Experiment","subscription_id":"2c48c51c-bd47-40d4-abbe-fb8eabd19c8c","resource_group":"aml-quickstarts-231015","workspace_name":"quick-starts-ws-231015","region":"westus2"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

In [24]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: Project_3_Experiment,
Id: AutoML_bc06420d-c0d5-4168-893b-65dae5508651_38,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta...
                                             PreFittedSoftVotingRegressor(estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmregressor', LightGBMRegressor(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('24', Pipeline(memory=None, steps=[('standardscalerwrapper',

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [57]:
model = best_run.register_model(
    model_name='Behavioral_Model', 
    model_path='outputs/model.pkl', 
    description='AutoML Model for Behavioral Dataset')

In [58]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-231015', subscription_id='2c48c51c-bd47-40d4-abbe-fb8eabd19c8c', resource_group='aml-quickstarts-231015'), name=Behavioral_Model, id=Behavioral_Model:2, version=2, tags={}, properties={})

In [28]:
best_run.download_files(output_directory='automl_output_dir')

In [47]:
env = Environment.from_conda_specification("AutoML-env", "automl_output_dir/outputs/conda_env_v_1_0_0.yml")
inference_config = InferenceConfig(entry_script='automl_output_dir/outputs/scoring_file_v_2_0_0.py',
                                   environment=env)

In [49]:
deployment_config = AciWebservice.deploy_configuration(
    enable_app_insights=True,
    auth_enabled=True) 

In [ ]:
service = Model.deploy(ws, "behavioral-model-endpoint", [model], inference_config, deployment_config, overwrite=True)

In [61]:
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-04-13 21:49:02+00:00 Creating Container Registry if not exists..
2023-04-13 21:59:02+00:00 Registering the environment.
2023-04-13 21:59:03+00:00 Building image.

: 

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [ ]:
rest_endpoint = service.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "behavioral-data-rest-endpoint"}
                        )

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

TODO: In the cell below, print the logs of the web service and delete the service

# Cleanup

In [ ]:
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
